In [1]:
import pandas as pd
import numpy as np

In [2]:
generic_drug = pd.read_csv('data/5year_generic_drug_us.csv', low_memory=False) # common generic drugs from year 2013 to year 2017

In [3]:
part_d_df_2017 = pd.read_csv('data/part_d_2017.csv', low_memory=False)
exclusion_df_2017 = pd.read_csv('data/exclusion_list_2018.csv', low_memory=False)
print("The data has a size of {}".format(part_d_df_2017.shape))

The data has a size of (25209130, 21)


In [4]:
# reorganize data columns
part_d_df_2017 = part_d_df_2017[['npi', 'nppes_provider_city','nppes_provider_state', 'specialty_description', 'description_flag',\
                        'drug_name', 'generic_name', 'bene_count', 'total_claim_count','total_day_supply', \
                        'total_drug_cost', 'bene_count_ge65', 'total_claim_count_ge65', 'total_day_supply_ge65',\
                        'total_drug_cost_ge65']]

# assigning exclusion (False = 0, True = 1)
exclusion_npi_2017 = exclusion_df_2017['NPI'].values.tolist()

part_d_df_2017['is_excluded'] = 0
part_d_df_2017.loc[part_d_df_2017['npi'].isin(exclusion_npi_2017), 'is_excluded'] = 1

# get US data to analyze
states = ['AK', 'AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', \
          'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', \
          'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', \
          'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', \
          'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI',' WY']

part_d_df_2017 = part_d_df_2017[part_d_df_2017['nppes_provider_state'].isin(states)]

print("The data has a size of {}".format(part_d_df_2017.shape))

The data has a size of (24792693, 16)


In [5]:
part_d_df_2017['year'] = 2017

In [6]:
# use generic_name to merge two tables 
part_d_df_2017_part = pd.merge(part_d_df_2017, generic_drug, on="generic_name")
part_d_df_2017_part.head()

,npi,nppes_provider_city,nppes_provider_state,specialty_description,description_flag,drug_name,generic_name,bene_count,total_claim_count,total_day_supply,total_drug_cost,bene_count_ge65,total_claim_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,is_excluded,year
0,1861409989,METHUEN,MA,Internal Medicine,S,LOSARTAN-HYDROCHLOROTHIAZIDE,LOSARTAN/HYDROCHLOROTHIAZIDE,19.0,58,4920,1055.68,NaN,NaN,NaN,NaN,0,2017
1,1740482959,SUMMIT,NJ,Family Practice,S,LOSARTAN-HYDROCHLOROTHIAZIDE,LOSARTAN/HYDROCHLOROTHIAZIDE,NaN,36,1560,444.42,NaN,NaN,NaN,NaN,0,2017
2,1437445202,PEORIA,IL,Family Practice,S,LOSARTAN-HYDROCHLOROTHIAZIDE,LOSARTAN/HYDROCHLOROTHIAZIDE,16.0,38,3060,445.57,NaN,NaN,NaN,NaN,0,2017
3,1366502155,AUSTELL,GA,Internal Medicine,S,LOSARTAN-HYDROCHLOROTHIAZIDE,LOSARTAN/HYDROCHLOROTHIAZIDE,12.0,45,3150,750.51,NaN,NaN,NaN,NaN,0,2017
4,1720033970,CHAPEL HILL,NC,Infectious Disease,S,LOSARTAN-HYDROCHLOROTHIAZIDE,LOSARTAN/HYDROCHLOROTHIAZIDE,NaN,11,330,86.47,0.0,0.0,0.0,0.0,0,2017


In [7]:
# use the groupby() and agg() methods to create a new dataframe. use numpy sum method to populate values
npi_exclusion_year_df_2017 = part_d_df_2017_part.groupby(["npi","is_excluded","year"]).agg({"total_claim_count": np.sum})

In [8]:
npi_exclusion_year_df_2017.head()

,,,total_claim_count
npi,is_excluded,year,
1003000126,0,2017,385
1003000142,0,2017,1835
1003000167,0,2017,38
1003000282,0,2017,54
1003000407,0,2017,2263


In [9]:
# to merge the tables we need to flatten the indices.
npi_exclusion_year_df_2017.reset_index(level=["npi","is_excluded","year"], inplace=True)

In [10]:
npi_exclusion_year_df_2017.head()

,npi,is_excluded,year,total_claim_count
0,1003000126,0,2017,385
1,1003000142,0,2017,1835
2,1003000167,0,2017,38
3,1003000282,0,2017,54
4,1003000407,0,2017,2263


In [11]:
# User pandas crosstab function to create a dataframe where each row is a prescriber identified by their NPI, 
# the columns are the drug generic names, the values are the sum of total_claim_count
npi_generic_cross_df_2017 = pd.crosstab(part_d_df_2017_part["npi"], part_d_df_2017_part["generic_name"], values=part_d_df_2017_part["total_claim_count"], 
                                  aggfunc=np.sum)

In [12]:
# To replace the NaN (not a number) from the cells with zero the fillna() method is used.
npi_generic_cross_df_2017 = npi_generic_cross_df_2017.fillna(0)

In [13]:
npi_generic_cross_df_2017.head()

generic_name,0.9 % SODIUM CHLORIDE,AA 5 %/CALCIUM/LYTES/DEXT 20 %,ABACAVIR SULFATE,ABACAVIR SULFATE/LAMIVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,ABATACEPT,ABATACEPT/MALTOSE,ABIRATERONE ACETATE,ABOBOTULINUMTOXINA,ACAMPROSATE CALCIUM,...,ZANAMIVIR,ZIDOVUDINE,ZILEUTON,ZIPRASIDONE HCL,ZIPRASIDONE MESYLATE,ZOLEDRONIC ACID,ZOLMITRIPTAN,ZOLPIDEM TARTRATE,ZONISAMIDE,ZOSTER VACCINE LIVE/PF
npi,,,,,,,,,,,,,,,,,,,,,
1003000126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000407,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# use npi to merge two tables npi_exclusion_year_df and npi_generic_cross_df
npi_exclusion_year_generic_df_2017 = pd.merge(npi_exclusion_year_df_2017, npi_generic_cross_df_2017.reset_index(level=["npi"]), on="npi")

In [15]:
npi_exclusion_year_generic_df_2017.head()

,npi,is_excluded,year,total_claim_count,0.9 % SODIUM CHLORIDE,AA 5 %/CALCIUM/LYTES/DEXT 20 %,ABACAVIR SULFATE,ABACAVIR SULFATE/LAMIVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,ABATACEPT,...,ZANAMIVIR,ZIDOVUDINE,ZILEUTON,ZIPRASIDONE HCL,ZIPRASIDONE MESYLATE,ZOLEDRONIC ACID,ZOLMITRIPTAN,ZOLPIDEM TARTRATE,ZONISAMIDE,ZOSTER VACCINE LIVE/PF
0,1003000126,0,2017,385,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1003000142,0,2017,1835,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003000167,0,2017,38,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1003000282,0,2017,54,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1003000407,0,2017,2263,18.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
len(npi_exclusion_year_generic_df_2017)

901177

In [ ]:
# npi_exclusion_year_generic_df_2017.to_csv('npi_exclusion_generic_2017_us.csv')

In [1]:
import pandas as pd
import numpy as np

In [2]:
generic_drug = pd.read_csv('data/5year_generic_drug_us.csv', low_memory=False) # common generic drugs from year 2013 to year 2017

In [3]:
part_d_df_2016 = pd.read_csv('data/part_d_2016.csv', low_memory=False)
exclusion_df_2016 = pd.read_csv('data/exclusion_list_2017.csv', low_memory=False)
print("The data has a size of {}".format(part_d_df_2016.shape))

The data has a size of (24964300, 21)


In [4]:
# reorganize data columns
part_d_df_2016 = part_d_df_2016[['npi', 'nppes_provider_city','nppes_provider_state', 'specialty_description', 'description_flag',\
                        'drug_name', 'generic_name', 'bene_count', 'total_claim_count','total_day_supply', \
                        'total_drug_cost', 'bene_count_ge65', 'total_claim_count_ge65', 'total_day_supply_ge65',\
                        'total_drug_cost_ge65']]

# assigning exclusion (False = 0, True = 1)
exclusion_npi_2016 = exclusion_df_2016['NPI'].values.tolist()

part_d_df_2016['is_excluded'] = 0
part_d_df_2016.loc[part_d_df_2016['npi'].isin(exclusion_npi_2016), 'is_excluded'] = 1

# get US data to analyze
states = ['AK', 'AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', \
          'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', \
          'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', \
          'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', \
          'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI',' WY']

part_d_df_2016 = part_d_df_2016[part_d_df_2016['nppes_provider_state'].isin(states)]

print("The data has a size of {}".format(part_d_df_2016.shape))

The data has a size of (24548475, 16)


In [5]:
part_d_df_2016['year'] = 2016

In [6]:
# use generic_name to merge two tables 
part_d_df_2016_part = pd.merge(part_d_df_2016, generic_drug, on="generic_name")
part_d_df_2016_part.head()

,npi,nppes_provider_city,nppes_provider_state,specialty_description,description_flag,drug_name,generic_name,bene_count,total_claim_count,total_day_supply,total_drug_cost,bene_count_ge65,total_claim_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,is_excluded,year
0,1104855030,IRON MOUNTAIN,MI,Hematology,S,AZITHROMYCIN,AZITHROMYCIN,11.0,14,70,76.90,NaN,NaN,NaN,NaN,0,2016
1,1083058937,NIAGARA FALLS,NY,Family Practice,S,AZITHROMYCIN,AZITHROMYCIN,21.0,21,105,126.72,NaN,NaN,NaN,NaN,0,2016
2,1063653673,PONTIAC,MI,Emergency Medicine,S,AZITHROMYCIN,AZITHROMYCIN,15.0,15,75,106.02,NaN,NaN,NaN,NaN,0,2016
3,1447297064,BOSTON,MA,Internal Medicine,S,AZITHROMYCIN,AZITHROMYCIN,15.0,16,83,102.12,NaN,NaN,NaN,NaN,0,2016
4,1376580506,BOOTHWYN,PA,Internal Medicine,S,AZITHROMYCIN,AZITHROMYCIN,50.0,66,303,578.53,NaN,NaN,NaN,NaN,0,2016


In [7]:
# use the groupby() and agg() methods to create a new dataframe. use numpy sum method to populate values
npi_exclusion_year_df_2016 = part_d_df_2016_part.groupby(["npi","is_excluded","year"]).agg({"total_claim_count": np.sum})

In [8]:
npi_exclusion_year_df_2016.head()

,,,total_claim_count
npi,is_excluded,year,
1003000126,0,2016,310
1003000142,0,2016,1633
1003000167,0,2016,32
1003000282,0,2016,26
1003000407,0,2016,1708


In [9]:
# to merge the tables we need to flatten the indices.
npi_exclusion_year_df_2016.reset_index(level=["npi","is_excluded","year"], inplace=True)

In [10]:
npi_exclusion_year_df_2016.head()

,npi,is_excluded,year,total_claim_count
0,1003000126,0,2016,310
1,1003000142,0,2016,1633
2,1003000167,0,2016,32
3,1003000282,0,2016,26
4,1003000407,0,2016,1708


In [11]:
# User pandas crosstab function to create a dataframe where each row is a prescriber identified by their NPI, 
# the columns are the drug generic names, the values are the sum of total_claim_count
npi_generic_cross_df_2016 = pd.crosstab(part_d_df_2016_part["npi"], part_d_df_2016_part["generic_name"], values=part_d_df_2016_part["total_claim_count"], 
                                  aggfunc=np.sum)

In [12]:
# To replace the NaN (not a number) from the cells with zero the fillna() method is used.
npi_generic_cross_df_2016 = npi_generic_cross_df_2016.fillna(0)

In [13]:
npi_generic_cross_df_2016.head()

generic_name,0.9 % SODIUM CHLORIDE,AA 5 %/CALCIUM/LYTES/DEXT 20 %,ABACAVIR SULFATE,ABACAVIR SULFATE/LAMIVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,ABATACEPT,ABATACEPT/MALTOSE,ABIRATERONE ACETATE,ABOBOTULINUMTOXINA,ACAMPROSATE CALCIUM,...,ZANAMIVIR,ZIDOVUDINE,ZILEUTON,ZIPRASIDONE HCL,ZIPRASIDONE MESYLATE,ZOLEDRONIC ACID,ZOLMITRIPTAN,ZOLPIDEM TARTRATE,ZONISAMIDE,ZOSTER VACCINE LIVE/PF
npi,,,,,,,,,,,,,,,,,,,,,
1003000126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# use npi to merge two tables npi_exclusion_year_df and npi_generic_cross_df
npi_exclusion_year_generic_df_2016 = pd.merge(npi_exclusion_year_df_2016, npi_generic_cross_df_2016.reset_index(level=["npi"]), on="npi")

In [15]:
npi_exclusion_year_generic_df_2016.head()

,npi,is_excluded,year,total_claim_count,0.9 % SODIUM CHLORIDE,AA 5 %/CALCIUM/LYTES/DEXT 20 %,ABACAVIR SULFATE,ABACAVIR SULFATE/LAMIVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,ABATACEPT,...,ZANAMIVIR,ZIDOVUDINE,ZILEUTON,ZIPRASIDONE HCL,ZIPRASIDONE MESYLATE,ZOLEDRONIC ACID,ZOLMITRIPTAN,ZOLPIDEM TARTRATE,ZONISAMIDE,ZOSTER VACCINE LIVE/PF
0,1003000126,0,2016,310,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1003000142,0,2016,1633,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003000167,0,2016,32,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1003000282,0,2016,26,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1003000407,0,2016,1708,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
len(npi_exclusion_year_generic_df_2016)

876761

In [17]:
#npi_exclusion_year_generic_df_2016.to_csv('npi_exclusion_generic_2016_us.csv')

In [1]:
import pandas as pd
import numpy as np

In [2]:
generic_drug = pd.read_csv('data/5year_generic_drug_us.csv', low_memory=False) # common generic drugs from year 2013 to year 2017

In [3]:
part_d_df_2015 = pd.read_csv('data/part_d_2015.csv', low_memory=False)
exclusion_df_2015 = pd.read_csv('data/exclusion_list_2016.csv', low_memory=False)
print("The data has a size of {}".format(part_d_df_2015.shape))

The data has a size of (24524894, 21)


In [4]:
# reorganize data columns
part_d_df_2015 = part_d_df_2015[['npi', 'nppes_provider_city','nppes_provider_state', 'specialty_description', 'description_flag',\
                        'drug_name', 'generic_name', 'bene_count', 'total_claim_count','total_day_supply', \
                        'total_drug_cost', 'bene_count_ge65', 'total_claim_count_ge65', 'total_day_supply_ge65',\
                        'total_drug_cost_ge65']]

# assigning exclusion (False = 0, True = 1)
exclusion_npi_2015 = exclusion_df_2015['NPI'].values.tolist()

part_d_df_2015['is_excluded'] = 0
part_d_df_2015.loc[part_d_df_2015['npi'].isin(exclusion_npi_2015), 'is_excluded'] = 1

# get US data to analyze
states = ['AK', 'AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', \
          'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', \
          'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', \
          'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', \
          'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI',' WY']

part_d_df_2015 = part_d_df_2015[part_d_df_2015['nppes_provider_state'].isin(states)]

print("The data has a size of {}".format(part_d_df_2015.shape))

The data has a size of (24101572, 16)


In [5]:
part_d_df_2015['year'] = 2015

In [6]:
# use generic_name to merge two tables 
part_d_df_2015_part = pd.merge(part_d_df_2015, generic_drug, on="generic_name")
part_d_df_2015_part.head()

,npi,nppes_provider_city,nppes_provider_state,specialty_description,description_flag,drug_name,generic_name,bene_count,total_claim_count,total_day_supply,total_drug_cost,bene_count_ge65,total_claim_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,is_excluded,year
0,1003000522,DELTONA,FL,Family Practice,S,GEMFIBROZIL,GEMFIBROZIL,NaN,27,2250,581.96,NaN,NaN,NaN,NaN,0,2015
1,1003000530,QUAKERTOWN,PA,Internal Medicine,S,GEMFIBROZIL,GEMFIBROZIL,NaN,21,635,382.23,NaN,NaN,NaN,NaN,0,2015
2,1003000902,LOUISVILLE,KY,Family Practice,S,GEMFIBROZIL,GEMFIBROZIL,NaN,37,1470,513.45,NaN,NaN,NaN,NaN,0,2015
3,1003002817,DORAL,FL,Family Practice,S,GEMFIBROZIL,GEMFIBROZIL,NaN,34,1020,238.62,NaN,13.0,390.0,102.43,0,2015
4,1003006552,FORT MEADE,FL,Family Practice,S,GEMFIBROZIL,GEMFIBROZIL,29.0,76,4904,1453.97,NaN,59.0,3734.0,1035.17,0,2015


In [7]:
# use the groupby() and agg() methods to create a new dataframe. use numpy sum method to populate values
npi_exclusion_year_df_2015 = part_d_df_2015_part.groupby(["npi","is_excluded","year"]).agg({"total_claim_count": np.sum})

In [8]:
npi_exclusion_year_df_2015.head()

,,,total_claim_count
npi,is_excluded,year,
1003000126,0,2015,546
1003000142,0,2015,1618
1003000167,0,2015,28
1003000282,0,2015,15
1003000407,0,2015,917


In [9]:
# to merge the tables we need to flatten the indices.
npi_exclusion_year_df_2015.reset_index(level=["npi","is_excluded","year"], inplace=True)

In [10]:
npi_exclusion_year_df_2015.head()

,npi,is_excluded,year,total_claim_count
0,1003000126,0,2015,546
1,1003000142,0,2015,1618
2,1003000167,0,2015,28
3,1003000282,0,2015,15
4,1003000407,0,2015,917


In [11]:
# User pandas crosstab function to create a dataframe where each row is a prescriber identified by their NPI, 
# the columns are the drug generic names, the values are the sum of total_claim_count
npi_generic_cross_df_2015 = pd.crosstab(part_d_df_2015_part["npi"], part_d_df_2015_part["generic_name"], values=part_d_df_2015_part["total_claim_count"], 
                                  aggfunc=np.sum)

In [12]:
# To replace the NaN (not a number) from the cells with zero the fillna() method is used.
npi_generic_cross_df_2015 = npi_generic_cross_df_2015.fillna(0)

In [13]:
npi_generic_cross_df_2015.head()

generic_name,0.9 % SODIUM CHLORIDE,AA 5 %/CALCIUM/LYTES/DEXT 20 %,ABACAVIR SULFATE,ABACAVIR SULFATE/LAMIVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,ABATACEPT,ABATACEPT/MALTOSE,ABIRATERONE ACETATE,ABOBOTULINUMTOXINA,ACAMPROSATE CALCIUM,...,ZANAMIVIR,ZIDOVUDINE,ZILEUTON,ZIPRASIDONE HCL,ZIPRASIDONE MESYLATE,ZOLEDRONIC ACID,ZOLMITRIPTAN,ZOLPIDEM TARTRATE,ZONISAMIDE,ZOSTER VACCINE LIVE/PF
npi,,,,,,,,,,,,,,,,,,,,,
1003000126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# use npi to merge two tables npi_exclusion_year_df and npi_generic_cross_df
npi_exclusion_year_generic_df_2015 = pd.merge(npi_exclusion_year_df_2015, npi_generic_cross_df_2015.reset_index(level=["npi"]), on="npi")

In [15]:
npi_exclusion_year_generic_df_2015.head()

,npi,is_excluded,year,total_claim_count,0.9 % SODIUM CHLORIDE,AA 5 %/CALCIUM/LYTES/DEXT 20 %,ABACAVIR SULFATE,ABACAVIR SULFATE/LAMIVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,ABATACEPT,...,ZANAMIVIR,ZIDOVUDINE,ZILEUTON,ZIPRASIDONE HCL,ZIPRASIDONE MESYLATE,ZOLEDRONIC ACID,ZOLMITRIPTAN,ZOLPIDEM TARTRATE,ZONISAMIDE,ZOSTER VACCINE LIVE/PF
0,1003000126,0,2015,546,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1003000142,0,2015,1618,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003000167,0,2015,28,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1003000282,0,2015,15,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1003000407,0,2015,917,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
len(npi_exclusion_year_generic_df_2015)

850074

In [17]:
# npi_exclusion_year_generic_df_2015.to_csv('npi_exclusion_generic_2015_us.csv')

In [1]:
import pandas as pd
import numpy as np

In [2]:
generic_drug = pd.read_csv('data/5year_generic_drug_us.csv', low_memory=False) # common generic drugs from year 2013 to year 2017

In [3]:
part_d_df_2014 = pd.read_csv('data/part_d_2014.csv', low_memory=False)
exclusion_df_2014 = pd.read_csv('data/exclusion_list_2015.csv', low_memory=False)
print("The data has a size of {}".format(part_d_df_2014.shape))

The data has a size of (24120618, 21)


In [4]:
# reorganize data columns
part_d_df_2014 = part_d_df_2014[['npi', 'nppes_provider_city','nppes_provider_state', 'specialty_description', 'description_flag',\
                        'drug_name', 'generic_name', 'bene_count', 'total_claim_count','total_day_supply', \
                        'total_drug_cost', 'bene_count_ge65', 'total_claim_count_ge65', 'total_day_supply_ge65',\
                        'total_drug_cost_ge65']]

# assigning exclusion (False = 0, True = 1)
exclusion_npi_2014 = exclusion_df_2014['NPI'].values.tolist()

part_d_df_2014['is_excluded'] = 0
part_d_df_2014.loc[part_d_df_2014['npi'].isin(exclusion_npi_2014), 'is_excluded'] = 1

# get US data to analyze
states = ['AK', 'AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', \
          'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', \
          'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', \
          'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', \
          'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI',' WY']

part_d_df_2014 = part_d_df_2014[part_d_df_2014['nppes_provider_state'].isin(states)]

print("The data has a size of {}".format(part_d_df_2014.shape))

The data has a size of (23697142, 16)


In [5]:
part_d_df_2014['year'] = 2014

In [6]:
# use generic_name to merge two tables 
part_d_df_2014_part = pd.merge(part_d_df_2014, generic_drug, on="generic_name")
part_d_df_2014_part.head()

,npi,nppes_provider_city,nppes_provider_state,specialty_description,description_flag,drug_name,generic_name,bene_count,total_claim_count,total_day_supply,total_drug_cost,bene_count_ge65,total_claim_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,is_excluded,year
0,1003046624,DOVER,DE,Internal Medicine,S,PANTOPRAZOLE SODIUM,PANTOPRAZOLE SODIUM,NaN,30,2160,828.14,NaN,18.0,1080.0,632.50,0,2014
1,1003046913,OCEAN CITY,NJ,Nurse Practitioner,S,PANTOPRAZOLE SODIUM,PANTOPRAZOLE SODIUM,12.0,34,2970,775.07,12.0,34.0,2970.0,775.07,0,2014
2,1003047523,HARLINGEN,TX,Physician Assistant,S,PANTOPRAZOLE SODIUM,PANTOPRAZOLE SODIUM,93.0,312,10428,3437.63,75.0,237.0,8058.0,2577.88,0,2014
3,1003047564,OMAHA,NE,Internal Medicine,S,PANTOPRAZOLE SODIUM,PANTOPRAZOLE SODIUM,16.0,41,1140,489.81,NaN,NaN,NaN,NaN,0,2014
4,1003047788,BENNINGTON,VT,Nurse Practitioner,S,PANTOPRAZOLE SODIUM,PANTOPRAZOLE SODIUM,NaN,16,600,379.66,NaN,NaN,NaN,NaN,0,2014


In [7]:
# use the groupby() and agg() methods to create a new dataframe. use numpy sum method to populate values
npi_exclusion_year_df_2014 = part_d_df_2014_part.groupby(["npi","is_excluded","year"]).agg({"total_claim_count": np.sum})

In [8]:
npi_exclusion_year_df_2014.head()

,,,total_claim_count
npi,is_excluded,year,
1003000126,0,2014,373
1003000142,0,2014,853
1003000167,0,2014,28
1003000407,0,2014,160
1003000423,0,2014,42


In [9]:
# to merge the tables we need to flatten the indices.
npi_exclusion_year_df_2014.reset_index(level=["npi","is_excluded","year"], inplace=True)

In [10]:
npi_exclusion_year_df_2014.head()

,npi,is_excluded,year,total_claim_count
0,1003000126,0,2014,373
1,1003000142,0,2014,853
2,1003000167,0,2014,28
3,1003000407,0,2014,160
4,1003000423,0,2014,42


In [11]:
# User pandas crosstab function to create a dataframe where each row is a prescriber identified by their NPI, 
# the columns are the drug generic names, the values are the sum of total_claim_count
npi_generic_cross_df_2014 = pd.crosstab(part_d_df_2014_part["npi"], part_d_df_2014_part["generic_name"], values=part_d_df_2014_part["total_claim_count"], 
                                  aggfunc=np.sum)

In [12]:
# To replace the NaN (not a number) from the cells with zero the fillna() method is used.
npi_generic_cross_df_2014 = npi_generic_cross_df_2014.fillna(0)

In [13]:
npi_generic_cross_df_2014.head()

generic_name,0.9 % SODIUM CHLORIDE,AA 5 %/CALCIUM/LYTES/DEXT 20 %,ABACAVIR SULFATE,ABACAVIR SULFATE/LAMIVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,ABATACEPT,ABATACEPT/MALTOSE,ABIRATERONE ACETATE,ABOBOTULINUMTOXINA,ACAMPROSATE CALCIUM,...,ZANAMIVIR,ZIDOVUDINE,ZILEUTON,ZIPRASIDONE HCL,ZIPRASIDONE MESYLATE,ZOLEDRONIC ACID,ZOLMITRIPTAN,ZOLPIDEM TARTRATE,ZONISAMIDE,ZOSTER VACCINE LIVE/PF
npi,,,,,,,,,,,,,,,,,,,,,
1003000126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# use npi to merge two tables npi_exclusion_year_df and npi_generic_cross_df
npi_exclusion_year_generic_df_2014 = pd.merge(npi_exclusion_year_df_2014, npi_generic_cross_df_2014.reset_index(level=["npi"]), on="npi")

In [15]:
npi_exclusion_year_generic_df_2014.head()

,npi,is_excluded,year,total_claim_count,0.9 % SODIUM CHLORIDE,AA 5 %/CALCIUM/LYTES/DEXT 20 %,ABACAVIR SULFATE,ABACAVIR SULFATE/LAMIVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,ABATACEPT,...,ZANAMIVIR,ZIDOVUDINE,ZILEUTON,ZIPRASIDONE HCL,ZIPRASIDONE MESYLATE,ZOLEDRONIC ACID,ZOLMITRIPTAN,ZOLPIDEM TARTRATE,ZONISAMIDE,ZOSTER VACCINE LIVE/PF
0,1003000126,0,2014,373,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1003000142,0,2014,853,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003000167,0,2014,28,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1003000407,0,2014,160,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1003000423,0,2014,42,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
len(npi_exclusion_year_generic_df_2014)

821365

In [17]:
# npi_exclusion_year_generic_df_2014.to_csv('npi_exclusion_generic_2014_us.csv')

In [1]:
import pandas as pd
import numpy as np

In [2]:
generic_drug = pd.read_csv('data/5year_generic_drug_us.csv', low_memory=False) # common generic drugs from year 2013 to year 2017

In [3]:
part_d_df_2013 = pd.read_csv('data/part_d_2013.csv', low_memory=False)
exclusion_df_2013 = pd.read_csv('data/exclusion_list_2014.csv', low_memory=False)
print("The data has a size of {}".format(part_d_df_2013.shape))

The data has a size of (23645873, 21)


In [4]:
# reorganize data columns
part_d_df_2013 = part_d_df_2013[['npi', 'nppes_provider_city','nppes_provider_state', 'specialty_description', 'description_flag',\
                        'drug_name', 'generic_name', 'bene_count', 'total_claim_count','total_day_supply', \
                        'total_drug_cost', 'bene_count_ge65', 'total_claim_count_ge65', 'total_day_supply_ge65',\
                        'total_drug_cost_ge65']]

# assigning exclusion (False = 0, True = 1)
exclusion_npi_2013 = exclusion_df_2013['NPI'].values.tolist()

part_d_df_2013['is_excluded'] = 0
part_d_df_2013.loc[part_d_df_2013['npi'].isin(exclusion_npi_2013), 'is_excluded'] = 1

# get US data to analyze
states = ['AK', 'AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', \
          'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', \
          'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', \
          'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', \
          'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI',' WY']

part_d_df_2013 = part_d_df_2013[part_d_df_2013['nppes_provider_state'].isin(states)]

print("The data has a size of {}".format(part_d_df_2013.shape))

The data has a size of (23233763, 16)


In [5]:
part_d_df_2013['year'] = 2013

In [6]:
# use generic_name to merge two tables 
part_d_df_2013_part = pd.merge(part_d_df_2013, generic_drug, on="generic_name")
part_d_df_2013_part.head()

,npi,nppes_provider_city,nppes_provider_state,specialty_description,description_flag,drug_name,generic_name,bene_count,total_claim_count,total_day_supply,total_drug_cost,bene_count_ge65,total_claim_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,is_excluded,year
0,1003049925,GLENNS FERRY,ID,Nurse Practitioner,S,NEXIUM,ESOMEPRAZOLE MAGNESIUM,NaN,16,720,5353.41,NaN,NaN,NaN,NaN,0,2013
1,1003050337,BLUE SPRINGS,MO,Family Practice,S,NEXIUM,ESOMEPRAZOLE MAGNESIUM,NaN,29,546,4064.01,NaN,NaN,NaN,NaN,0,2013
2,1003050675,CAPE GIRARDEAU,MO,Nurse Practitioner,S,NEXIUM,ESOMEPRAZOLE MAGNESIUM,NaN,21,930,6415.59,NaN,21.0,930.0,6415.59,0,2013
3,1003051087,BRONX,NY,Internal Medicine,S,NEXIUM,ESOMEPRAZOLE MAGNESIUM,15.0,44,1655,11677.42,NaN,NaN,NaN,NaN,0,2013
4,1003051699,STUART,FL,Internal Medicine,S,NEXIUM,ESOMEPRAZOLE MAGNESIUM,NaN,19,1600,11040.55,NaN,NaN,NaN,NaN,0,2013


In [7]:
# use the groupby() and agg() methods to create a new dataframe. use numpy sum method to populate values
npi_exclusion_year_df_2013 = part_d_df_2013_part.groupby(["npi","is_excluded","year"]).agg({"total_claim_count": np.sum})

In [8]:
npi_exclusion_year_df_2013.head()

,,,total_claim_count
npi,is_excluded,year,
1003000126,0,2013,128
1003000142,0,2013,407
1003000167,0,2013,34
1003000282,0,2013,14
1003000407,0,2013,206


In [9]:
# to merge the tables we need to flatten the indices.
npi_exclusion_year_df_2013.reset_index(level=["npi","is_excluded","year"], inplace=True)

In [10]:
npi_exclusion_year_df_2013.head()

,npi,is_excluded,year,total_claim_count
0,1003000126,0,2013,128
1,1003000142,0,2013,407
2,1003000167,0,2013,34
3,1003000282,0,2013,14
4,1003000407,0,2013,206


In [11]:
# User pandas crosstab function to create a dataframe where each row is a prescriber identified by their NPI, 
# the columns are the drug generic names, the values are the sum of total_claim_count
npi_generic_cross_df_2013 = pd.crosstab(part_d_df_2013_part["npi"], part_d_df_2013_part["generic_name"], values=part_d_df_2013_part["total_claim_count"], 
                                  aggfunc=np.sum)

In [12]:
# To replace the NaN (not a number) from the cells with zero the fillna() method is used.
npi_generic_cross_df_2013 = npi_generic_cross_df_2013.fillna(0)

In [13]:
npi_generic_cross_df_2013.head()

generic_name,0.9 % SODIUM CHLORIDE,AA 5 %/CALCIUM/LYTES/DEXT 20 %,ABACAVIR SULFATE,ABACAVIR SULFATE/LAMIVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,ABATACEPT,ABATACEPT/MALTOSE,ABIRATERONE ACETATE,ABOBOTULINUMTOXINA,ACAMPROSATE CALCIUM,...,ZANAMIVIR,ZIDOVUDINE,ZILEUTON,ZIPRASIDONE HCL,ZIPRASIDONE MESYLATE,ZOLEDRONIC ACID,ZOLMITRIPTAN,ZOLPIDEM TARTRATE,ZONISAMIDE,ZOSTER VACCINE LIVE/PF
npi,,,,,,,,,,,,,,,,,,,,,
1003000126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003000407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# use npi to merge two tables npi_exclusion_year_df and npi_generic_cross_df
npi_exclusion_year_generic_df_2013 = pd.merge(npi_exclusion_year_df_2013, npi_generic_cross_df_2013.reset_index(level=["npi"]), on="npi")

In [15]:
npi_exclusion_year_generic_df_2013.head()

,npi,is_excluded,year,total_claim_count,0.9 % SODIUM CHLORIDE,AA 5 %/CALCIUM/LYTES/DEXT 20 %,ABACAVIR SULFATE,ABACAVIR SULFATE/LAMIVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,ABATACEPT,...,ZANAMIVIR,ZIDOVUDINE,ZILEUTON,ZIPRASIDONE HCL,ZIPRASIDONE MESYLATE,ZOLEDRONIC ACID,ZOLMITRIPTAN,ZOLPIDEM TARTRATE,ZONISAMIDE,ZOSTER VACCINE LIVE/PF
0,1003000126,0,2013,128,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1003000142,0,2013,407,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003000167,0,2013,34,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1003000282,0,2013,14,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1003000407,0,2013,206,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
len(npi_exclusion_year_generic_df_2013)

791878

In [17]:
# npi_exclusion_year_generic_df_2013.to_csv('npi_exclusion_generic_2013_us.csv')

In [1]:
import pandas as pd

In [2]:
npi_exclusion_generic_2017_us_df = pd.read_csv('npi_exclusion_generic_2017_us.csv', low_memory=False)
npi_exclusion_generic_2016_us_df = pd.read_csv('npi_exclusion_generic_2016_us.csv', low_memory=False)
# npi_exclusion_generic_2017_us_df = pd.read_csv('npi_exclusion_generic_2015_us.csv', low_memory=False)
# npi_exclusion_generic_2016_us_df = pd.read_csv('npi_exclusion_generic_2014_us.csv', low_memory=False)
# npi_exclusion_generic_2016_us_df = pd.read_csv('npi_exclusion_generic_2013_us.csv', low_memory=False)

In [3]:
npi_exclusion_generic_us_df_union_all= pd.concat([npi_exclusion_generic_2017_us_df, npi_exclusion_generic_2016_us_df])
# npi_exclusion_generic_us_df_union_all= pd.concat([npi_exclusion_generic_2017_us_df, npi_exclusion_generic_2016_us_df,npi_exclusion_generic_2015_us_df,npi_exclusion_generic_2014_us_df,npi_exclusion_generic_2013_us_df])

In [4]:
len(npi_exclusion_generic_us_df_union_all)

1777938

In [6]:
npi_exclusion_generic_us_df_union_all.sort_values('npi').head()

,Unnamed: 0,npi,is_excluded,year,total_claim_count,0.9 % SODIUM CHLORIDE,AA 5 %/CALCIUM/LYTES/DEXT 20 %,ABACAVIR SULFATE,ABACAVIR SULFATE/LAMIVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,...,ZANAMIVIR,ZIDOVUDINE,ZILEUTON,ZIPRASIDONE HCL,ZIPRASIDONE MESYLATE,ZOLEDRONIC ACID,ZOLMITRIPTAN,ZOLPIDEM TARTRATE,ZONISAMIDE,ZOSTER VACCINE LIVE/PF
0,0,1003000126,0,2017,385,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0,1003000126,0,2016,310,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1003000142,0,2016,1633,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1003000142,0,2017,1835,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1003000167,0,2016,32,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# npi_exclusion_generic_us_df_union_all.sort_values('npi').to_csv('npi_exclusion_generic_us_sortbyNPI.csv')